In [65]:
import numpy as np
import matplotlib.pyplot as plt2
from operator import add
from IPython.display import Audio
from IPython.display import display
from scipy.fft import fft, ifft,fftfreq
from matplotlib.ticker import FormatStrFormatter

In [66]:
%matplotlib widget

#Create Plot grid
fig= plt2.figure(figsize=(10, 12), constrained_layout=True)
widths = [9,1]
heights = [4,4,4,4]
gs=fig.add_gridspec(4,2,width_ratios=widths, height_ratios=heights, wspace=0.05)

ax1=fig.add_subplot(gs[0,:1])
ax2=fig.add_subplot(gs[1,:1])
ax3=fig.add_subplot(gs[2,:1])
ax4=fig.add_subplot(gs[3,:1])

ax1.clear()
ax2.clear()
ax3.clear()
ax4.clear()

#Create waveform: sin(ax)*exp(b*x^2)
def wave(time, ampl, freq, pos, width):
    wave=ampl*np.sin(time*2*np.pi*freq)*np.exp(-1*(((pos-time)/(width))**2))
    return wave

#SampleRate kHz (also determines max. frequency 3 kHz is good compromise). Totaltime in seconds.
totaltime=10
samplerate=3

#Number of notes per second. Free to change number of waves WN
WN=3

#Mean note width [seconds]. 
PWT=2.5

#Tonal system or continous frequency mode
tonal=True

#Harmonics on/of (on: True, off: False, capital letter!), number of harmonics e.g.: 10.
harm=True
harmo=5

#Time array
samples=samplerate*1000*totaltime
time=np.linspace(0,totaltime,samples,endpoint=True)

#Set total summation array to zero
yt=[0]*samples

#Random wave addition
for w in range(int(WN*totaltime)):
    
    #Determine startin position wave packet
    pos=np.random.randint(1,samples)/(samplerate*1000)
    
    #Random generator note width. Shorter notes more likely
    a=np.array([2,2,2,2,4,4,4,8,8,16])
    am=np.mean(a)
    l=np.random.choice(a)
    w=PWT*l/am
    width=np.random.randint(1,w*100)/(100*3)
    
    #Determine ampltitude wave packet.
    ampl=np.random.randint(10,100)
    
    #Set harmonic array to zero.
    fth=[0]*samples
    
    #Select tone system continues or notes.
    if tonal==True:
        nmax=12*np.log(0.5*samplerate*1000/440)/np.log(2)
        n=np.random.randint(-40,int(nmax))
        
        freq=440*(2**(n/12))
    else:
        freq=np.random.randint(20,samplerate*1000*0.5)
    
    #Create basic wave
    ft=wave(time,ampl,freq,pos,width)
    
    #Create harmonics
    if harm==True:
        for h in range(harmo):
            
            harmonic=freq*np.random.randint(2,12)
            ampl=ampl*np.random.randint(1,100)/100
            fth=wave(time,ampl,harmonic,pos,width)
            ft=list(map(add, ft, fth))
    
    yt=list(map(add, yt, ft))

    #Plot waves order widest on back
    wmax=PWT*(np.max(a))/am
    wmin=PWT*(np.min(a))/am
    order=int(-(WN*totaltime)*width/(wmax-wmin))
    ax1.plot(time, ft, linewidth='0.1',zorder=order)
    
#Plot graphs time signal
ax1.set_title('Time Signal $f(t)$')
ax1.set_xlabel('time (s)')
ax1.set_ylabel('f(t)')
ax1.grid(b=True, which='major', color='#666666', linestyle='-')

#Plot Total time signal summation
ax2.plot(time, yt, linewidth='0.1')
ax2.set_title('Total Time Signal $f(t)$')
ax2.set_xlabel('time (s)')
ax2.set_ylabel('f(t)')
ax2.grid(b=True, which='major', color='#666666', linestyle='-')

#Fourier Transform FFT
yf=fft(yt)
yfam=np.abs(yf)/np.sum(np.abs(yf))
yffphase=np.angle(yf)
xf=fftfreq(samples, 1/samples)/totaltime
xf=np.sort(xf)

#Plot freuency graphs amplitude and phase
ax3.plot(xf, yfam,linewidth='0.5')
ax3.set_title('Amplitude, $F(\omega)$')
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('Amplitude')
ax3.grid(b=True, which='major', color='#666666', linestyle='-')
ax3.axis([0,1000*samplerate*0.5,0,1.05*np.max(yfam)])

ax4.plot(xf, yffphase, marker='.', linestyle='', markersize=0.25)
ax4.set_title('Phase, $F(\omega)$')
ax4.set_xlabel('Frequency (Hz)')
ax4.set_ylabel('Phase')
ax4.grid(b=True, which='major', color='#666666', linestyle='-')
ax4.axis([0,1000*samplerate*0.5,1.05*np.min(np.angle(yf)),1.2*np.max(np.angle(yf))])

ax1.text(1.02,0.98,'Rate: ' + str(samplerate) +  ' [kHz]\nTime: ' + str(totaltime)  + ' [s]\n\nDensity: ' +    str(WN) + ' [notes/s]\nMean: ' + str(PWT) + ' [s/note]\n\nHarmonics: '+ str(harmo)  + '\nActive: ' +  str(harm) + '\n\nTonal: ' + str(tonal), backgroundcolor='white', transform=ax1.transAxes, fontsize=10,verticalalignment='top', zorder=5)

display(Audio(yt, rate=1000*samplerate))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …